In [1]:
!pip install numpy pandas matplotlib seaborn
!pip install scikit-learn
!pip install jinja2
!pip install sentence-transformers hdbscan

In [2]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import rcParams
from matplotlib import font_manager as fm
# pd.options.display.max_columns = None
# pd.options.display.max_rows = None

In [ ]:
# 데이터 불러오기
df = pd.read_csv('/Users/c-market/Downloads/data_list.csv', low_memory=False)
df_ct = pd.read_csv('/Users/c-market/Downloads/df_new_with_depths_v6.csv')

In [4]:
# 전처리 코드
#
#

# 원본은 냅두라고
df_test1 = df.copy()

def preprocess_pdt_name(name):
    name = str(name).lower()            # 소문자
    name = re.sub(r'[()/\\\[\]]', ' ', name)
  # 괄호, 슬래시, 역슬래시 제거 → 공백
    name = re.sub(r'\s+', ' ', name)    # 연속 공백을 한 칸으로
    return name.strip()                  # 앞뒤 공백 제거

df_test1['pdt_name_clean'] = df_test1['pdt_name'].apply(preprocess_pdt_name)

df_new = df_test1[['pdt_name','brand_name','pdt_name_clean','pdt_cas','pdt_spec']]

df_new
# # 데이터 추출
# data.to_csv('/Users/c-market/Downloads/data.csv')

,pdt_name,brand_name,pdt_name_clean,pdt_cas,pdt_spec
0,전산기록지(80컬럼/양미싱),오피스디포,전산기록지 80컬럼 양미싱,NaN,NaN
1,전산기록지(132컬럼/양미싱),오피스디포,전산기록지 132컬럼 양미싱,NaN,NaN
2,팩스용지(15mX210mm/한솔),오피스디포,팩스용지 15mx210mm 한솔,NaN,NaN
3,팩스용지(15mX216mm/한솔),오피스디포,팩스용지 15mx216mm 한솔,NaN,NaN
4,팩스용지(30mX210mm/한솔),오피스디포,팩스용지 30mx210mm 한솔,NaN,NaN
...,...,...,...,...,...
104483,BKA-001 무릎 보호대 K,NaN,bka-001 무릎 보호대 k,NaN,NaN
104484,BKH-002 손목 보호대,NaN,bkh-002 손목 보호대,NaN,NaN
104485,[고려에이스] Centrifuge Tube Rack 스텐원심관랙,고려에이스,고려에이스 centrifuge tube rack 스텐원심관랙,NaN,NaN
104486,Bambu 저온용 재사용 가능 스풀,뱀부랩,bambu 저온용 재사용 가능 스풀,NaN,ea / 필라멘트는 재사용 가능 스풀 미포함입니다.


In [ ]:
df_new.describe()
# df_new.to_csv('/Users/c-market/Downloads/df_new.csv')

,pdt_name,brand_name,pdt_name_clean,pdt_cas,pdt_spec
count,104487,73748,104488,40554,27257
unique,101448,6392,101211,17288,23394
top,삭제,오피스디포,삭제,mixture,상세규격 별도기재
freq,98,14566,98,216,241


In [10]:
import re
import numpy as np

# CAS 번호 패턴 (예: 64-17-5)
cas_pat = r"\b\d{2,7}-\d{2}-\d\b"

def extract_all_cas(text: str):
    found = re.findall(cas_pat, str(text))
    return sorted(set(x.strip() for x in found))

# pdt_cas 빈값(NaN, "") 처리
df_new["pdt_cas"] = df_new["pdt_cas"].replace("", np.nan)

# pdt_cas가 비어 있는 행에 한해 pdt_name에서 CAS 추출 후 채우기
mask_empty = df_new["pdt_cas"].isna()
df_new.loc[mask_empty, "pdt_cas"] = (
    df_new.loc[mask_empty, "pdt_name"]
          .apply(lambda x: "|".join(extract_all_cas(x)) or np.nan)
)

# 확인
df_new

# # pdt_cas가 NaN인 행만 추출
# data = df_new[df_new['pdt_cas'].isna()]
# # pdt_cas 가 NaN인 것들을 뽑아내면 시약과 분리되어 데이터가 더 가벼워짐

/var/folders/vh/v6yfwz_x4xz31663ql80v00r0000gq/T/ipykernel_87315/3762492176.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["pdt_cas"] = df_new["pdt_cas"].replace("", np.nan)


,pdt_name,brand_name,pdt_name_clean,pdt_cas,pdt_spec
0,전산기록지(80컬럼/양미싱),오피스디포,전산기록지 80컬럼 양미싱,NaN,NaN
1,전산기록지(132컬럼/양미싱),오피스디포,전산기록지 132컬럼 양미싱,NaN,NaN
2,팩스용지(15mX210mm/한솔),오피스디포,팩스용지 15mx210mm 한솔,NaN,NaN
3,팩스용지(15mX216mm/한솔),오피스디포,팩스용지 15mx216mm 한솔,NaN,NaN
4,팩스용지(30mX210mm/한솔),오피스디포,팩스용지 30mx210mm 한솔,NaN,NaN
...,...,...,...,...,...
104483,BKA-001 무릎 보호대 K,NaN,bka-001 무릎 보호대 k,NaN,NaN
104484,BKH-002 손목 보호대,NaN,bkh-002 손목 보호대,NaN,NaN
104485,[고려에이스] Centrifuge Tube Rack 스텐원심관랙,고려에이스,고려에이스 centrifuge tube rack 스텐원심관랙,NaN,NaN
104486,Bambu 저온용 재사용 가능 스풀,뱀부랩,bambu 저온용 재사용 가능 스풀,NaN,ea / 필라멘트는 재사용 가능 스풀 미포함입니다.


In [12]:
# pdt_name과 갱신된 pdt_cas 확인 (앞부분 20개만)
print(df_new[["pdt_name", "pdt_cas"]].head(20))

# pdt_cas가 새로 채워진 행만 확인하고 싶다면:
updated_rows = df_new[df_new["pdt_cas"].str.contains("-", na=False)]
print(updated_rows[["pdt_name", "pdt_cas"]].head(20))

# 전체 통계 요약 (비어있던 값 몇 개 채워졌는지 확인)
print(df_new["pdt_cas"].isna().sum(), "rows still empty")
print(df_new["pdt_cas"].notna().sum(), "rows now filled")


                             pdt_name pdt_cas
0                     전산기록지(80컬럼/양미싱)     NaN
1                    전산기록지(132컬럼/양미싱)     NaN
2                  팩스용지(15mX210mm/한솔)     NaN
3                  팩스용지(15mX216mm/한솔)     NaN
4                  팩스용지(30mX210mm/한솔)     NaN
5                  팩스용지(30mX216mm/한솔)     NaN
6         복사용지A4(80g/더블에이/500매X5권/박스)     NaN
7     컬러레이저용지A4 (90g/500매/NEUSIEDLER)     NaN
8     컬러레이저용지A3 (90g/500매/NEUSIEDLER)     NaN
9         복사용지B4(80g/더블에이/500매X5권/박스)     NaN
10        복사용지B5(80g/더블에이/500매X5권/박스)     NaN
11   컬러레이저용지A4 (100g/500매/NEUSIEDLER)     NaN
12   컬러레이저용지A4 (120g/250매/NEUSIEDLER)     NaN
13   컬러레이저용지A3 (120g/250매/NEUSIEDLER)     NaN
14   컬러레이저용지A4 (160g/250매/NEUSIEDLER)     NaN
15   컬러레이저용지A3 (160g/250매/NEUSIEDLER)     NaN
16    컬러레이저용지A4(100g/100매/NEUSIEDLER)     NaN
17    컬러레이저용지A4(120g/100매/NEUSIEDLER)     NaN
18   컬러레이저용지A3 (100g/500매/NEUSIEDLER)     NaN
19  복사용지A4(75g/SMARTCOPY/250매X10권/박스)     NaN
                                  

In [15]:
# pdt_name이 '삭제'인 행 제거
df_new = df_new[df_new["pdt_name"] != "삭제"].copy()

# 인덱스 리셋
df_new = df_new.reset_index(drop=True)



In [ ]:
# df_new.to_csv('/Users/c-market/Downloads/df_new.csv')

In [ ]:
df_new # cas 번호 채우기 + pdt_name이 '삭제'인 행 지우기 전처리 완료

,pdt_name,brand_name,pdt_name_clean,pdt_cas,pdt_spec
0,전산기록지(80컬럼/양미싱),오피스디포,전산기록지 80컬럼 양미싱,NaN,NaN
1,전산기록지(132컬럼/양미싱),오피스디포,전산기록지 132컬럼 양미싱,NaN,NaN
2,팩스용지(15mX210mm/한솔),오피스디포,팩스용지 15mx210mm 한솔,NaN,NaN
3,팩스용지(15mX216mm/한솔),오피스디포,팩스용지 15mx216mm 한솔,NaN,NaN
4,팩스용지(30mX210mm/한솔),오피스디포,팩스용지 30mx210mm 한솔,NaN,NaN
...,...,...,...,...,...
104385,BKA-001 무릎 보호대 K,NaN,bka-001 무릎 보호대 k,NaN,NaN
104386,BKH-002 손목 보호대,NaN,bkh-002 손목 보호대,NaN,NaN
104387,[고려에이스] Centrifuge Tube Rack 스텐원심관랙,고려에이스,고려에이스 centrifuge tube rack 스텐원심관랙,NaN,NaN
104388,Bambu 저온용 재사용 가능 스풀,뱀부랩,bambu 저온용 재사용 가능 스풀,NaN,ea / 필라멘트는 재사용 가능 스풀 미포함입니다.


In [ ]:
import pandas as pd

# v5 불러오기
df_v5 = pd.read_csv("/Users/c-market/Downloads/df_new_with_depths_v5.csv.csv")

# 브랜드명 없는 경우에만 후보 확인
missing_brand = df_v5[df_v5["브랜드명"].isna()][
    ["pdt_name", "pdt_name_clean", "브랜드명_후보", "근거_토큰"]
]

# 샘플 20행 출력
print(missing_brand.head(20))

# 특정 후보 브랜드만 보고 싶을 때 (예: "삼성")
samsung_candidates = df_v5[df_v5["브랜드명_후보"] == "삼성"][
    ["pdt_name", "pdt_name_clean", "근거_토큰"]
]
print(samsung_candidates.head(20))


,pdt_name,brand_name,pdt_name_clean,pdt_cas,pdt_spec,브랜드명,브랜드명_후보,근거_토큰,규격/스펙,color_code_candidates,code_token_candidates,__added_paper,__added_qty,__added_hycode,__added_paper_glued
0,전산기록지(80컬럼/양미싱),오피스디포,전산기록지 80컬럼 양미싱,NaN,NaN,오피스디포,NaN,NaN,80컬럼,[],[],False,False,False,False
1,전산기록지(132컬럼/양미싱),오피스디포,전산기록지 132컬럼 양미싱,NaN,NaN,오피스디포,NaN,NaN,132컬럼,[],[],False,False,False,False
2,팩스용지(15mX210mm/한솔),오피스디포,팩스용지 15mx210mm 한솔,NaN,NaN,오피스디포,NaN,NaN,15mx210mm,[],['15mx210mm'],False,False,False,False
3,팩스용지(15mX216mm/한솔),오피스디포,팩스용지 15mx216mm 한솔,NaN,NaN,오피스디포,NaN,NaN,15mx216mm,[],['15mx216mm'],False,False,False,False
4,팩스용지(30mX210mm/한솔),오피스디포,팩스용지 30mx210mm 한솔,NaN,NaN,오피스디포,NaN,NaN,30mx210mm,[],['30mx210mm'],False,False,False,False
5,팩스용지(30mX216mm/한솔),오피스디포,팩스용지 30mx216mm 한솔,NaN,NaN,오피스디포,NaN,NaN,30mx216mm,[],['30mx216mm'],False,False,False,False
6,복사용지A4(80g/더블에이/500매X5권/박스),오피스디포,복사용지a4 80g 더블에이 500매x5권 박스,NaN,NaN,오피스디포,NaN,NaN,80g/500매x5권/a4,[],['80g'],False,True,False,True
7,컬러레이저용지A4 (90g/500매/NEUSIEDLER),오피스디포,컬러레이저용지a4 90g 500매 neusiedler,NaN,NaN,오피스디포,NaN,NaN,90g/500매/a4,[],['90g'],False,False,False,True
8,컬러레이저용지A3 (90g/500매/NEUSIEDLER),오피스디포,컬러레이저용지a3 90g 500매 neusiedler,NaN,NaN,오피스디포,NaN,NaN,90g/500매/a3,[],['90g'],False,False,False,True
9,복사용지B4(80g/더블에이/500매X5권/박스),오피스디포,복사용지b4 80g 더블에이 500매x5권 박스,NaN,NaN,오피스디포,NaN,NaN,80g/500매x5권/b4,[],['80g'],False,True,False,True
